In [7]:
!pip install opendatasets

In [8]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/prasoonkottarathil/polycystic-ovary-syndrome-pcos")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: janeetajolly
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/prasoonkottarathil/polycystic-ovary-syndrome-pcos


100%|██████████| 120k/120k [00:00<00:00, 261MB/s]

In [9]:
import pandas as pd
xlsx_path = "polycystic-ovary-syndrome-pcos/PCOS_data_without_infertility.xlsx"
xls = pd.ExcelFile(xlsx_path)
print(xls.sheet_names)

['Instructions', 'Full_new']


In [10]:
df = pd.read_excel(xlsx_path, sheet_name='Full_new')
print(df.shape)
df.head()

(541, 45)


,Sl. No,Patient File No.,PCOS (Y/N),Age (yrs),Weight (Kg),Height(Cm),BMI,Blood Group,Pulse rate(bpm),RR (breaths/min),...,Fast food (Y/N),Reg.Exercise(Y/N),BP _Systolic (mmHg),BP _Diastolic (mmHg),Follicle No. (L),Follicle No. (R),Avg. F size (L) (mm),Avg. F size (R) (mm),Endometrium (mm),Unnamed: 44
0,1,1,0,28,44.6,152.0,19.300000,15,78,22,...,1.0,0,110,80,3,3,18.0,18.0,8.5,NaN
1,2,2,0,36,65.0,161.5,24.921163,15,74,20,...,0.0,0,120,70,3,5,15.0,14.0,3.7,NaN
2,3,3,1,33,68.8,165.0,25.270891,11,72,18,...,1.0,0,120,80,13,15,18.0,20.0,10.0,NaN
3,4,4,0,37,65.0,148.0,29.674945,13,72,20,...,0.0,0,120,70,2,2,15.0,14.0,7.5,NaN
4,5,5,0,25,52.0,161.0,20.060954,11,72,18,...,0.0,0,120,80,3,4,16.0,14.0,7.0,NaN


In [11]:
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace('\n', '', regex=True)
df = df.drop(columns=["Sl. No", "Patient File No."], errors="ignore")

In [12]:
df = df.drop(columns=["Sl. No", "Patient File No."], errors="ignore")

In [13]:
print(df.columns.tolist())

['PCOS (Y/N)', 'Age (yrs)', 'Weight (Kg)', 'Height(Cm)', 'BMI', 'Blood Group', 'Pulse rate(bpm)', 'RR (breaths/min)', 'Hb(g/dl)', 'Cycle(R/I)', 'Cycle length(days)', 'Marraige Status (Yrs)', 'Pregnant(Y/N)', 'No. of aborptions', 'I   beta-HCG(mIU/mL)', 'II    beta-HCG(mIU/mL)', 'FSH(mIU/mL)', 'LH(mIU/mL)', 'FSH/LH', 'Hip(inch)', 'Waist(inch)', 'Waist:Hip Ratio', 'TSH (mIU/L)', 'AMH(ng/mL)', 'PRL(ng/mL)', 'Vit D3 (ng/mL)', 'PRG(ng/mL)', 'RBS(mg/dl)', 'Weight gain(Y/N)', 'hair growth(Y/N)', 'Skin darkening (Y/N)', 'Hair loss(Y/N)', 'Pimples(Y/N)', 'Fast food (Y/N)', 'Reg.Exercise(Y/N)', 'BP _Systolic (mmHg)', 'BP _Diastolic (mmHg)', 'Follicle No. (L)', 'Follicle No. (R)', 'Avg. F size (L) (mm)', 'Avg. F size (R) (mm)', 'Endometrium (mm)', 'Unnamed: 44']


In [14]:
selected_columns = [
    'Age (yrs)', 'Weight (Kg)', 'Height(Cm)', 'BMI',
    'Cycle(R/I)', 'Fast food (Y/N)', 'Pimples(Y/N)',
    'hair growth(Y/N)', 'Skin darkening (Y/N)',
    'Follicle No. (L)', 'Follicle No. (R)',
    'Avg. F size (L) (mm)', 'Avg. F size (R) (mm)',
    'PCOS (Y/N)'
]
df = df[selected_columns]

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# Separate X and y
X = df.drop(columns=['PCOS (Y/N)'])
y = df['PCOS (Y/N)']

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Save model and scaler
joblib.dump(model, "pcos_model.pkl")
joblib.dump(scaler, "scaler.pkl")


[[71  6]
 [ 7 25]]
              precision    recall  f1-score   support

           0       0.91      0.92      0.92        77
           1       0.81      0.78      0.79        32

    accuracy                           0.88       109
   macro avg       0.86      0.85      0.85       109
weighted avg       0.88      0.88      0.88       109



['scaler.pkl']

In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Use same model config as before
rf_model = RandomForestClassifier(random_state=42)

# Use scaled features
scores = cross_val_score(rf_model, X_scaled, y, cv=5, scoring='accuracy')

print("Cross-validation accuracies:", scores)
print("Mean Accuracy:", np.mean(scores))
print("Standard Deviation:", np.std(scores))


Cross-validation accuracies: [0.88990826 0.90740741 0.86111111 0.9537037  0.80555556]
Mean Accuracy: 0.8835372069317025
Standard Deviation: 0.049229628766757766


In [17]:
from sklearn.model_selection import cross_validate

results = cross_validate(
    rf_model, X_scaled, y, cv=5,
    scoring=['accuracy', 'f1', 'precision', 'recall']
)

for metric in results:
    print(metric, results[metric])


fit_time [0.17370224 0.1796174  0.16597366 0.33155107 0.32057309]
score_time [0.01646209 0.01682734 0.02419233 0.03510904 0.08114052]
test_accuracy [0.88990826 0.90740741 0.86111111 0.9537037  0.80555556]
test_f1 [0.82352941 0.86111111 0.76190476 0.92753623 0.67692308]
test_precision [0.875      0.83783784 0.85714286 0.94117647 0.75862069]
test_recall [0.77777778 0.88571429 0.68571429 0.91428571 0.61111111]


In [18]:
from google.colab import files

files.download('pcos_model.pkl')
files.download('scaler.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>